In [1]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage,ToolMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os

print("✅ All imports successful")

c:\Users\User\Desktop\Working_with_LLMS\llmvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All imports successful


In [2]:
# Load environment variables
load_dotenv()
openai_api_key = os.getenv("openai_key")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded successfully")

✅ API key loaded successfully


In [14]:

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

PDF_PATH = "../data/tax_bills_pdfs"
CHROMA_PATH = "chroma_db"

loader = DirectoryLoader(
    PDF_PATH,
    glob="**/*.pdf",
    loader_cls=PyPDFLoader
)

documents = loader.load()
print(f"✅ Loaded {len(documents)} pages from PDFs")

print(documents)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=120
)

chunks = splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=openai_api_key
)
vectorstore = Chroma(
    collection_name="nigeria_tax_bills",
    embedding_function=embeddings,
    persist_directory=CHROMA_PATH
)

vectorstore.add_documents(chunks)

print(f"Ingested {len(chunks)} chunks")


✅ Loaded 375 pages from PDFs
[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-07-15T19:32:44-07:00', 'author': 'Computer Section', 'moddate': '2025-07-15T19:32:44-07:00', 'title': 'C:\\Users\\Computer Section\\Deskt', 'source': '..\\data\\tax_bills_pdfs\\JOINT_REVENUE_BOARD_ACT_2025.pdf', 'total_pages': 36, 'page': 0, 'page_label': '1'}, page_content='2025  No. 6          A  347Joint Revenue Board of Nigeria\n(Establishment) Act, 2025\nFederal Republic of Nigeria\nOfficial Gazette\nNo. 117 Lagos—26th  June, 2025 Vol. 112\nGovernment Notice  No. 25\nThe following is published as supplement to this Gazette :\nAct No. Short Title Page\n6  Joint Revenue Board of Nigeria (Establishment) Act, 2025 .. A349-382\nExtraordinary\nPrinted and Published by The Federal Government Printer, Lagos, Nigeria\nFGP 29/72025/500\nAnnual Subscription from 1st January, 2025 is Local : N25,000.00 Overseas : N37,500.00 [Surface Mail]\nN49,500.00 [Second Class 

In [15]:
from duckduckgo_search import DDGS

@tool
def retrieve_tax_documents(query: str) -> str:
    """
    Retrieve official Nigerian tax policy documents.
    """
    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5, "fetch_k": 12}
    )

    docs = retriever.invoke(query)

    if not docs:
        return "No relevant official documents found."

    return "\n\n---\n\n".join(
        f"Source: {doc.metadata.get('source', 'Official Document')}\n{doc.page_content}"
        for doc in docs
    )


@tool
def calculator(expression:str)-> str:
    """
    Evaluate a mathematical expression and return the result.Use this toool when you need to perform calculations.

    Args:
        expression: A mathematical expression like "2 + 2" or "15*37"01_Introduction_to_LangGraph.ipynb

    Returns:
        The calculated result as a string

    Examples:
    - "2 + 2" returns "4"
    - "100 / 5" returns "20.0"
    - "2 ** 10" returns "1024"

    """
    try:
        #Evaluate the expression safely
        result=eval(expression,{"__builtins__": {}},{})
        return str(result)
    
    except Exception as e:
        return f"Error calculating: {str(e)}"
    
print("Calculator tool created")




TRUSTED_DOMAINS = [
    "oecd.org",
    "imf.org",
    "worldbank.org",
    "gov.za",
    "irs.gov",
    "firs.gov.ng",
    "finance.gov.ng"
]

@tool
def restricted_policy_search(query: str) -> str:
    """
    Search ONLY trusted government and policy websites
    for legal and policy comparisons.
    """
    domain_filter = " OR ".join([f"site:{d}" for d in TRUSTED_DOMAINS])
    search_query = f"{query} {domain_filter}"

    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(search_query, max_results=5):
            results.append(
                f"Title: {r['title']}\n"
                f"Source: {r['href']}\n"
                f"Snippet: {r['body']}\n"
            )

    if not results:
        return "No trusted sources found."

    return "\n\n".join(results)


Calculator tool created


In [19]:
from typing import Literal

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")
system_prompt = SystemMessage(content="""
You are TaxBotNG, an AI assistant helping Nigerians understand the 2024 Nigerian Tax Reform Bills.

RULES:
- Use retrieval ONLY for questions about tax policy, VAT, PAYE, businesses, or state revenue.
- NEVER guess or hallucinate.
- If documents do not contain the answer, say so.
- Explain answers in simple Nigerian-friendly English.
- Cite sources when retrieval is used.

DO NOT retrieve for greetings, math, dates, or general knowledge.
""")

tools = [retrieve_tax_documents,calculator,restricted_policy_search]
llm_with_tools = llm.bind_tools(tools)



✅ LLM initialized: gpt-4o-mini


In [20]:
def assistant(state: MessagesState):
    messages = [system_prompt] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    last = state["messages"][-1]
    return "tools" if last.tool_calls else "__end__"



In [21]:
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "__end__": END}
)
builder.add_edge("tools", "assistant")

agent = builder.compile(checkpointer=MemorySaver())


In [22]:
def run_agent(user_input: str,thread_id: str = "test_session"):
    """
    Run the agent and display the conversation.
    """
    print(f"\n{'='*70}")
    print(f"👤 User: {user_input}")
    print(f"{'='*70}\n")


    result = agent.invoke(
        {"messages":[HumanMessage(content=user_input)]},
        config={"configurable":{"thread_id":thread_id}}
    )

    for message in result["messages"]:
        if isinstance(message,HumanMessage):
            continue
        elif isinstance(message,AIMessage):
            if message.tool_calls:
                print(f"🤖 Agent: [Calling tool: {message.tool_calls[0]['name']}]")
            else:
                print(f"🤖 Agent: {message.content}")
        elif isinstance(message, ToolMessage):
            print(f"🔧 Tool Result: {message.content[:100]}..." if len(message.content) > 100 else f"🔧 Tool Result: {message.content}")
    
    print(f"\n{'='*70}\n")
 
print("✅ Test function ready")

✅ Test function ready


In [ ]:
while True:
    userinput=input("User:")
    run_agent(user_input=userinput)


👤 User: what is 2+2

🤖 Agent: [Calling tool: calculator]
🔧 Tool Result: 4
🤖 Agent: The result of 2 + 2 is 4.



👤 User: how many percentage does the current tax law say i will pay

🤖 Agent: [Calling tool: calculator]
🔧 Tool Result: 4
🤖 Agent: The result of 2 + 2 is 4.
🤖 Agent: [Calling tool: retrieve_tax_documents]
🔧 Tool Result: Source: ..\data\tax_bills_pdfs\Nigeria-Tax-Act-2025.pdf
and above in the relevant year of assessment...
🤖 Agent: The current tax law specifies different tax rates for individuals and companies. 

1. **For Individuals**: The income tax payable is detailed in the Fourth Schedule of the Nigerian Tax Act. The exact percentage can vary based on income levels, so it's essential to check that specific schedule for precise rates.

2. **For Companies**: The effective tax rate for companies can be as high as 85% of their chargeable profits, depending on their activities and the nature of their business.

If you need more specific percentages based on your income or bus